# Trabalho 1 - Computação Gráfica
## Sergio Carrazzoni de Toledo Piza - 9361073
## João Villaça - 10724239


# Comandos:
- Movimente para a esquerda e direita utilizando as setas ou a/d
- Atire com a barra de espaço
- Para sair do jogo, aperta as teclas esc ou del

### Primeiro, vamos importar as bibliotecas necessárias.
Verifique no código anterior um script para instalar as dependências necessárias (OpenGL e GLFW) antes de prosseguir.

In [1]:
import glfw
from OpenGL.GL import *
import OpenGL.GL.shaders
from OpenGL.GLU import *
import numpy as np

### Inicializando janela

In [2]:
fullscreen = True

glfw.init()
glfw.window_hint(glfw.VISIBLE, glfw.FALSE);
if fullscreen:
    window = glfw.create_window(1920, 1080, "Space Invaders", glfw.get_primary_monitor(), None)
else:
    window = glfw.create_window(1920, 1080, "Space Invaders", None, None)

glfw.make_context_current(window)

### GLSL (OpenGL Shading Language)

Aqui veremos nosso primeiro código GLSL.

É uma linguagem de shading de alto nível baseada na linguagem de programação C.

Nós estamos escrevendo código GLSL como se "strings" de uma variável (mas podemos ler de arquivos texto). Esse código, depois, terá que ser compilado e linkado ao nosso programa. 

Iremos aprender GLSL conforme a necessidade do curso. Usarmos uma versão do GLSL mais antiga, compatível com muitos dispositivos.

### GLSL para Vertex Shader

No Pipeline programável, podemos interagir com Vertex Shaders.

No código abaixo, estamos fazendo o seguinte:

* Definindo uma variável chamada position do tipo vec2.
* Definindo uma variável chamada mat_transformation do tipo mat4 (matriz 4x4).
* Usamos vec2, pois nosso programa (na CPU) irá enviar apenas duas coordenadas para plotar um ponto. Podemos mandar três coordenadas (vec3) e até mesmo quatro coordenadas (vec4).
* void main() é o ponto de entrada do nosso programa (função principal)
* gl_Position é uma variável especial do GLSL. Variáveis que começam com 'gl_' são desse tipo. Nesse caso, determina a posição de um vértice. Observe que todo vértice tem 4 coordenadas, por isso nós combinamos nossa variável vec2 com uma variável vec4. Além disso, nós modificamos nosso vetor com base em uma matriz de transformação, conforme estudado na Aula05.

In [3]:
vertex_code = """
        attribute vec2 position;
        uniform mat4 mat_transformation;
        void main(){
            gl_Position = mat_transformation * vec4(position,0.0,1.0);
        }
        """

### GLSL para Fragment Shader

No Pipeline programável, podemos interagir com Fragment Shaders.

No código abaixo, estamos fazendo o seguinte:

* void main() é o ponto de entrada do nosso programa (função principal)
* gl_FragColor é uma variável especial do GLSL. Variáveis que começam com 'gl_' são desse tipo. Nesse caso, determina a cor de um fragmento. Nesse caso é um ponto, mas poderia ser outro objeto (ponto, linha, triangulos, etc).

### Possibilitando modificar a cor.

Nos exemplos anteriores, a variável gl_FragColor estava definida de forma fixa (com cor R=0, G=0, B=0).

Agora, nós vamos criar uma variável do tipo "uniform", de quatro posições (vec4), para receber o dado de cor do nosso programa rodando em CPU.

In [4]:
fragment_code = """
        uniform vec4 color;
        void main(){
            gl_FragColor = color;
        }
        """

### Requisitando slot para a GPU para nossos programas Vertex e Fragment Shaders

In [5]:
# Request a program and shader slots from GPU
program  = glCreateProgram()
vertex   = glCreateShader(GL_VERTEX_SHADER)
fragment = glCreateShader(GL_FRAGMENT_SHADER)


### Associando nosso código-fonte aos slots solicitados

In [6]:
# Set shaders source
glShaderSource(vertex, vertex_code)
glShaderSource(fragment, fragment_code)

### Compilando o Vertex Shader

Se há algum erro em nosso programa Vertex Shader, nosso app para por aqui.

In [7]:
# Compile shaders
glCompileShader(vertex)
if not glGetShaderiv(vertex, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(vertex).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Vertex Shader")


### Compilando o Fragment Shader

Se há algum erro em nosso programa Fragment Shader, nosso app para por aqui.

In [8]:
glCompileShader(fragment)
if not glGetShaderiv(fragment, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(fragment).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Fragment Shader")

### Associando os programas compilado ao programa principal

In [9]:
# Attach shader objects to the program
glAttachShader(program, vertex)
glAttachShader(program, fragment)


### Linkagem do programa

In [10]:
# Build program
glLinkProgram(program)
if not glGetProgramiv(program, GL_LINK_STATUS):
    print(glGetProgramInfoLog(program))
    raise RuntimeError('Linking error')
    
# Make program the default program
glUseProgram(program)

## Configuração dos inimigos

In [11]:
enemies_per_line = 11
enemy_lines = 4
enemy_total = enemies_per_line * enemy_lines
enemy_vcount = enemy_total * 4 # quantidade total de vértices utilizados pelos inimigos

# quantidade que será necessária dividir o tamanho da tela para caber todos os inimidos igualmente
division_factor = 2 * (enemies_per_line )

lateral_space = 0.25
vertical_space = 0.0

enemy_vertices = []

# preparando espaço para 3 vértices usando 2 coordenadas (x,y)
vertices = np.zeros(enemy_vcount + 7 + 4, [("position", np.float32, 2)])

# calculando os vértices dos inimidos dinamicamente
for j in range(0, enemy_lines):
    for i in range(1, enemies_per_line * 4):
        if i%2 == 1:
            vertices['position'][j * enemies_per_line * 4 + i - 1] = (
                ((1 - lateral_space)/division_factor) * i - 1 + lateral_space,
                1 - ((1/division_factor)) - (j * (lateral_space + vertical_space))
            )
            vertices['position'][j * enemies_per_line * 4 + i] = (
                ((1 - lateral_space)/division_factor) * i - 1 + lateral_space,
                1 - ((1/division_factor)) * 2 - (j * (lateral_space + vertical_space)) 
            )
            

#adicionando vértices dos players
vertices['position'][enemy_vcount + 0] = (-0.07, -0.98) # triangulo do player vertice 1
vertices['position'][enemy_vcount + 1] = (+0.07, -0.98) # triangulo do player vertice 2
vertices['position'][enemy_vcount + 2] = (+0.00, -0.85) # triangulo do player vertice 3
vertices['position'][enemy_vcount + 3] = (-0.07, -0.98) # retângulo do player vertice 1
vertices['position'][enemy_vcount + 4] = (+0.07, -0.98) # retângulo do player vertice 2
vertices['position'][enemy_vcount + 5] = (-0.07, -0.90) # retângulo do player vertice 3
vertices['position'][enemy_vcount + 6] = (+0.07, -0.90) # retângulo do player vertice 4
player_center_x = 0.0
player_center_y = (-0.98 - 0.85)/2


vertices['position'][enemy_vcount + 7] = (-0.002, -0.01) # vértice do retângulo do tiro 1
vertices['position'][enemy_vcount + 8] = (+0.002, -0.01) # vértice do retângulo do tiro 1
vertices['position'][enemy_vcount + 9] = (-0.002, +0.01) # vértice do retângulo do tiro 1
vertices['position'][enemy_vcount + 10] = (+0.002, +0.01) # vértice do retângulo do tiro 1

# for i in range(1, enemies_per_line * 4):
#     if i%2 == 1:
#         vertices['position'][i - 1] = (
#             ((1 - lateral_space)/division_factor) * i - 1 + lateral_space,
#             1 - ((1/division_factor))
#         )
#         vertices['position'][i] = (
#             ((1 - lateral_space)/division_factor) * i - 1 + lateral_space,
#             1 - ((1/division_factor)) * 2)

#print(vertices[0])
#print(vertices[1])
#vertices = np.delete(vertices, [0, 1])

#print(vertices[0])
#print(vertices[1])

In [12]:
'''
Detecta se um ponto está dentro dos vértices de um retângulo para a colisão dos tiros
v1 = vértice inferior esquerdo do retângulo
v2 = vértice superior direito do retângulo
x, y = coordenadas do ponto para checar
'''
def inside_square(v1, v2, x, y):
    if (x > v1[0] and x < v2[0] and y > v1[1] and y < v2[1]):
        return True
    return False

### Preparando dados para enviar a GPU

Nesse momento, nós compilamos nossos Vertex e Program Shaders para que a GPU possa processá-los.

Por outro lado, as informações de vértices geralmente estão na CPU e devem ser transmitidas para a GPU.


### Para enviar nossos dados da CPU para a GPU, precisamos requisitar um slot.

In [13]:
# Request a buffer slot from GPU
buffer = glGenBuffers(1)
# Make this buffer the default one
glBindBuffer(GL_ARRAY_BUFFER, buffer)


### Abaixo, nós enviamos todo o conteúdo da variável vertices.

Veja os parâmetros da função glBufferData [https://www.khronos.org/registry/OpenGL-Refpages/gl4/html/glBufferData.xhtml]

In [14]:
# Upload data
glBufferData(GL_ARRAY_BUFFER, vertices.nbytes, vertices, GL_DYNAMIC_DRAW)
glBindBuffer(GL_ARRAY_BUFFER, buffer)

### Associando variáveis do programa GLSL (Vertex Shaders) com nossos dados

Primeiro, definimos o byte inicial e o offset dos dados.

In [15]:
# Bind the position attribute
# --------------------------------------
stride = vertices.strides[0]
offset = ctypes.c_void_p(0)


Em seguida, soliciamos à GPU a localização da variável "position" (que guarda coordenadas dos nossos vértices). Nós definimos essa variável no Vertex Shader.

In [16]:
loc = glGetAttribLocation(program, "position")
glEnableVertexAttribArray(loc)

A partir da localização anterior, nós indicamos à GPU onde está o conteúdo (via posições stride/offset) para a variável position (aqui identificada na posição loc).

Outros parâmetros:

* Definimos que possui duas coordenadas
* Que cada coordenada é do tipo float (GL_FLOAT)
* Que não se deve normalizar a coordenada (False)

Mais detalhes: https://www.khronos.org/registry/OpenGL-Refpages/gl4/html/glVertexAttribPointer.xhtml

In [17]:
glVertexAttribPointer(loc, 2, GL_FLOAT, False, stride, offset)

###  Novidade aqui! Vamos pegar a localização da variável color (uniform) para que possamos alterá-la em nosso laço da janela!

In [18]:
loc_color = glGetUniformLocation(program, "color")
R = 1.0
G = 1.0
B = 1.0

In [19]:
def multiplica_matriz(a, b):
    m_a = a.reshape(4, 4)
    m_b = b.reshape(4, 4)
    m_c = np.dot(m_a, m_b)
    return m_c.reshape(1, 16)

### Capturando eventos de teclado e modificando variáveis para a matriz de transformação

In [20]:
game_should_close = 0
player_shoot = 0
bullet_position_x = 0
bullet_position_y = -1

# exemplo para matriz de translacao
t_x = 0

def key_event(window,key,scancode,action,mods):
    global t_x, game_should_close, player_shoot, bullet_position_x, bullet_position_y
    
    #print('[key event] key=',key)
#     print('[key event] scancode=',scancode)
#     print('[key event] action=',action)
#     print('[key event] mods=',mods)
#     print('-------')
    if key == 263 or key == 65: t_x -= 0.01 #movimenta o jogador para a esquerda com a tecla esquerda ou a tecla a
    if key == 262 or key == 68: t_x += 0.01 #movimenta o jogador para a direita com a tecla direita ou a tecla d
        
    if key == 256 or key == 261: # tecla esc or del fecham o jogo
        game_should_close = 1
        
    if key == 32 and not player_shoot: # barra de espaço atira
        player_shoot = 1
        bullet_position_x = player_center_x
        bullet_position_y = -0.95
        
    
glfw.set_key_callback(window,key_event)



### Loop principal da janela.
Enquanto a janela não for fechada, esse laço será executado. É neste espaço que trabalhamos com algumas interações com a OpenGL.

A novidade agora é a função glDrawArrays()

Tal função recebe o tipo de primitiva (GL_TRIANGLES), o índice inicial do array de vértices (vamos mostrar todos os três vértices, por isso começamos com 0) e a quantidade de vértices ( len(vertices) ).

In [21]:
enemy_max_shift = lateral_space # máximo que o inimigo pode andar em uma direção
enemy_direction = 1 # direção do inimigo (direita = 1, esquerda = -1)
enemy_speed = 0.00001 # velocidade com que o inimigo anda
enemy_initial_speed = np.copy(enemy_speed)
e_x = 0.0 # quantidade que o inimigo andou
e_y = 0.0 # altura em que o inimigo se encontra

bullet_rotation = 0
bullet_rotation_speed = 0.017
bullet_speed = 0.0007

killed_enemies = []
killed_enemies_scale = np.full(enemy_total, 1.00)
killed_enemies_rotation = np.zeros(enemy_total)
killed_enemies_x = np.zeros(enemy_total)
killed_enemies_y = np.zeros(enemy_total)

initial_vertices = np.copy(vertices)

glfw.show_window(window)
while not glfw.window_should_close(window) and not game_should_close:
    player_center_x = t_x
    
    glfw.poll_events() 

    e_x = e_x + (enemy_speed * enemy_direction)
    
    # quando o inimigo atingir uma dar bordas, a velocidade dele é adicionada com a velocidade inicial dividida por 2
    if (e_x > enemy_max_shift and enemy_direction) or (e_x < enemy_max_shift * -1 and enemy_direction == -1):
        e_y = e_y - 1/division_factor
        enemy_speed = enemy_speed + enemy_initial_speed/2

    # troca a direção do inimigo quando atingir uma das bordas
    if e_x > enemy_max_shift and enemy_direction == 1:
        enemy_direction = -1
    elif e_x < enemy_max_shift * -1 and enemy_direction == -1:
        enemy_direction = 1
        
    
    glClear(GL_COLOR_BUFFER_BIT) 
    glClearColor(0.0, 0.0, 0.0, 1.0)
    
    
    # DESENHANDO OS INIMIGOS
    glUniform4f(loc_color, 1.0, 1.0, 1.0, 1.0) ### modificando a cor do objeto!

    #Draw Triangle
    mat_translation = np.array([    1.0, 0.0, 0.0, e_x, 
                                    0.0, 1.0, 0.0, e_y, 
                                    0.0, 0.0, 1.0, 0.0, 
                                    0.0, 0.0, 0.0, 1.0], np.float32)
    
    loc = glGetUniformLocation(program, "mat_transformation")
    glUniformMatrix4fv(loc, 1, GL_TRUE, mat_translation)

    #adiciona os inimigos
    for i in range(0 , enemy_total):
        # se o inimigo já não estiver sido morto
        if i not in killed_enemies:
            v1 = np.copy(initial_vertices['position'][i * 4 + 1])
            v2 = np.copy(initial_vertices['position'][i * 4 + 2])
            v1[0] = v1[0] + e_x
            v2[0] = v2[0] + e_x
            # detecta a colisão entre o inimigo e a bala do jogador
            if (inside_square(v1, v2, bullet_position_x, bullet_position_y) and player_shoot):
                killed_enemies.append(i)
                killed_enemies_x[i] = e_x
                killed_enemies_y[i] = e_y
                player_shoot = 0
                
            # se não houver colisão entre eles, adiciona o inimigo na tela
            else:
                glDrawArrays(GL_TRIANGLE_STRIP, i * 4, 4)
                    

    
    # DESENHANDO OS INIMIGOS QUE FORAM MORTOS
    for i in killed_enemies:
        k_s = killed_enemies_scale[i]
        k_x = killed_enemies_x[i]
        k_y = killed_enemies_y[i]
        k_r = killed_enemies_rotation[i]
        if k_s > 0:
            killed_enemies_rotation[i] = killed_enemies_rotation[i] + 0.001
            killed_enemies_scale[i] = killed_enemies_scale[i] - 0.0008
            killed_enemies_y[i] = killed_enemies_y[i] - 0.001
            mat_scale = np.array([  k_s, 0.0, 0.0, 0.0, 
                                    0.0, k_s, 0.0, 0.0, 
                                    0.0, 0.0, 1.0, 0.0, 
                                    0.0, 0.0, 0.0, 1.0], np.float32)
            
            mat_translation = np.array([    1.0, 0.0, 0.0, k_x, 
                                            0.0, 1.0, 0.0, k_y, 
                                            0.0, 0.0, 1.0, 0.0, 
                                            0.0, 0.0, 0.0, 1.0], np.float32)
            
            mat_rotation = np.array([   np.cos(k_r), -np.sin(k_r), 0.0, 0.0, 
                                        np.sin(k_r),  np.cos(k_r), 0.0, 0.0, 
                                        0.0,          0.0,         1.0, 0.0, 
                                        0.0,          0.0,         0.0, 1.0], np.float32)
            
            
            mat_final = multiplica_matriz(mat_translation, mat_rotation)
            mat_final = multiplica_matriz(mat_final, mat_scale)
            
            loc = glGetUniformLocation(program, "mat_transformation")
            glUniformMatrix4fv(loc, 1, GL_TRUE, mat_final)
            glDrawArrays(GL_TRIANGLE_STRIP, i * 4, 4)
            
    
    
    # DESENHANDO O PLAYER
    glUniform4f(loc_color, 0.0, 1.0, 0.0, 1.0) ### modificando a cor do objeto!
    #impede o player de transladar para fora da tela
    if t_x > 1 - enemy_max_shift/2:
        t_x =  1 - enemy_max_shift/2
    if t_x < (1 - enemy_max_shift/2) * -1:
        t_x = -1 + enemy_max_shift/2
        
    mat_translation = np.array([    1.0, 0.0, 0.0, t_x, 
                                    0.0, 1.0, 0.0, 0.0, 
                                    0.0, 0.0, 1.0, 0.0, 
                                    0.0, 0.0, 0.0, 1.0], np.float32)
    
    loc = glGetUniformLocation(program, "mat_transformation")
    glUniformMatrix4fv(loc, 1, GL_TRUE, mat_translation)
    
    glDrawArrays(GL_TRIANGLES, enemy_vcount, 3)
    glDrawArrays(GL_TRIANGLE_STRIP, enemy_vcount + 3, 4)
    
    
    
    # DESENHANDO O TIRO
    if player_shoot and bullet_position_y > 1:
        player_shoot = 0
    
    if player_shoot:
        glUniform4f(loc_color, 0.0, 1.0, 0.0, 1.0) ### modificando a cor do objeto!
        bullet_position_y = bullet_position_y + bullet_speed
        bullet_rotation = bullet_rotation + bullet_rotation_speed

        mat_translation = np.array([    1.0, 0.0, 0.0, bullet_position_x, 
                                        0.0, 1.0, 0.0, bullet_position_y, 
                                        0.0, 0.0, 1.0, 0.0, 
                                        0.0, 0.0, 0.0, 1.0], np.float32)

        mat_rotation = np.array([       np.cos(bullet_rotation), -np.sin(bullet_rotation), 0.0, 0.0, 
                                        np.sin(bullet_rotation),  np.cos(bullet_rotation), 0.0, 0.0, 
                                        0.0,          0.0,         1.0, 0.0, 
                                        0.0,          0.0,         0.0, 1.0], np.float32)



        mat_final = multiplica_matriz(mat_translation, mat_rotation)

        loc = glGetUniformLocation(program, "mat_transformation")
        glUniformMatrix4fv(loc, 1, GL_TRUE, mat_final)

        glDrawArrays(GL_TRIANGLE_STRIP, enemy_vcount + 7, 4)

    
    glfw.swap_buffers(window)

glfw.terminate()